In [1]:
import math

In [ ]:
cot_prf_template = """ 
Answer the following query based on the context:

Context:{PRF doc 1}
{PRF doc 2}
{PRF doc 3}

Query:{query}

Give the rationale before answering
"""

In [2]:
def read_file(file):
    fp = open(file)
    output = dict()
    for line in fp.readlines():
        print(line.strip().split('\t'))
        id, text = line.strip().split('\t')
        output[id] = text
    return output

def example_list2dict(input):
    output = dict()
    for word in input.split():
        if output.get(word) is None:
            output[word] = 0
        output[word] += 1
    return output

def cal_idf(doc_dict):
    doc_num = len(doc_dict)
    idf = dict()
    for doc_id in doc_dict:
        doc_text = list(set(doc_dict[doc_id].split()))
        for word in doc_text:
            if idf.get(word) is None:
                idf[word] = 0
            idf[word] += 1
    for word in idf:
        idf[word] = math.log((doc_num - idf[word] + 0.5) / (idf[word] + 0.5))
    return idf

In [3]:
read_file("Pseudo-Relevance-Feedback/data/doc.txt")

["0'\\t'Jaguar is the luxury vehicle brand of Jaguar Land Rover, a British multinational car manufacturer with its headquarters in Whitley, Coventry England, owned by the Indian compnay Tata Motors."]


ValueError: not enough values to unpack (expected 2, got 1)

In [2]:
def bm25(query, doc, idf, avg_doc_len=374):
    k1 = 1.2
    k2 = 1
    b = 0.75
    score = 0.0
    for word in query:
        if doc.get(word) == None:
            continue
        W_i = idf[word]
        f_i = doc[word]
        qf_i = query[word]
        doc_len = sum(doc.values())
        K = k1 * (1 - b + b * doc_len / avg_doc_len)
        R1 = f_i * (k1 + 1) / (f_i + K)
        R2 = qf_i * (k2 + 1) / (qf_i + k2)
        R = R1 * R2
        score += W_i * R
    return score

In [ ]:
def GetScore(query, doc_name,doc_dict,idf):
    query = example_list2dict(query)
    doc = example_list2dict(doc_dict[doc_name])
    score = bm25(query, doc, idf)
    return score

In [ ]:
def find_prf(query, docs):
    doc_list = split(docs)
    prompt = "find the most relevant passage to the query"
    llm(prompt + doc_list)
    return doc

In [ ]:
def generate_cot_prf_prompt(query):
    PRF_doc_list = []
    
    for i in range(3):
        doc = find_prf(query, docs)
        docs = doc_delete(docs,doc)
        PRF_doc_list.append(doc)
        
    cot_prf_prompt = f""" 
    Answer the following query based on the context:

    Context:{PRF_doc_list[0]}
    {PRF_doc_list[1]}
    {PRF_doc_list[2]}

    Query:{query}

    Give the rationale before answering
    
    """
    return cot_prf_prompt